In [18]:
import numpy as np
import numpy.linalg as npl
import scipy.sparse as spsp

In [25]:
def training_set_creator(*args):
    """
    args are a non-zero number of lists of size 3.
    Example of use: we want the creation of an iteratable representing all possible elements on a n-dimensional grid
    first dimension goes from a to b with c elements, then you pass "[a,b,c]" as a first argument.
    return: n-dimensional grid on which you can iterate.
    
    """
    linspace_args = (np.linspace(arg[0],arg[1],arg[2]) for arg in args)
    
    meshes = np.meshgrid(*linspace_args)
    dimensions = [mesh.ravel() for mesh in meshes]
    
    tuples = zip(*dimensions)
    return list(tuples)
#     for tup in tuples:
#         yield(tuple(tup))

In [31]:
n = 5
N_diff = 2**n
f_diff = np.zeros(N_diff+2)
f_diff[0] = 1

#construction des matrices A en format sparse
tab_A_0 = [np.repeat([1,0],[N_diff//2,N_diff//2+1],0),np.repeat([0,-2,-1,0],[1,N_diff//2-1,1,N_diff//2+1],0),np.repeat([0,1,0],[1,N_diff//2-1,N_diff//2+1],0)]
tab_A_1 = [np.repeat([0,((N_diff+2)**2),0],[N_diff//2,N_diff//2,1],0),np.repeat([1,0,-((N_diff+2)**2),-2*((N_diff+2)**2),1],[1,N_diff//2-1,1,N_diff//2,1],0),np.repeat([0,((N_diff+2)**2)],[N_diff//2,N_diff//2+1],0)]
A_0 = spsp.diags(tab_A_0,[-1,0,1],(N_diff+2,N_diff+2))*((N_diff+2)**2)
A_1 = spsp.diags(tab_A_1,[-1,0,1],(N_diff+2,N_diff+2))


In [54]:
def apply_mu(mu):
    """
    from output of pre_computer and a mu: computes A_rb and f_rb and u_rb, the solution of A_rb.u_rb=f_rb
    """
    A = A_1.todense()
    A += A_0.todense() * mu[0]
    
    f = f_diff * mu[1]      
        
    return [A, f]

def to_array_A_and_f(MUs):
    A = []
    f = []
    
    for mu in MUs:
        temp = apply_mu(mu)
        A.append(temp[0])
        f.append(temp[1])
        
    return np.array(A), np.array(f)

In [77]:
def greedy_algorithm(tol, MUs):
    # On prend un mu (au hasard) pour initialiser l'algorithme
    mu_1 = MUs[0]
    A_delta_mu, f_delta_mu = apply_mu(mu_1)
#     print(A_delta_mu, f_delta_mu)
    u_delta_mu_1 = npl.solve(A_delta_mu, f_delta_mu)
    np.delete(MUs, 0)
    
    # B est la matrice de changement de base de A_delta à A_rb
    B = np.copy(u_delta_mu_1)
    
    # A @ u = f pour un mu donné
    
    # TODO: Repenser précomputing de A_DELTA et F_DELTA pour diminuer coûts mémoire
    A_DELTA, F_DELTA = to_array_A_and_f(MUs)    
    
    # On calcule tous les u_delta pour éviter de le refair dans la boucle par la suite
    U_DELTA = np.array([npl.solve(A_DELTA[idx], F_DELTA[idx]) for idx in range(len(MUs))])
    
    err = 1e10
    
    n = 1
    
    while err > tol and len(MUs) > 0:
        print(f"Itération n°{n}")
        
        eta = np.zeros(len(MUs))
        
        for idx_mu, mu in enumerate(MUs):
            A_delta_mu = A_DELTA[idx_mu]
            f_delta_mu = F_DELTA[idx_mu]
        
            u_delta_mu = U_DELTA[idx_mu]
            
            # A_rb_mu @ u_rb_mu = f_rb_mu
            # A_rb_mu = B.T @ A_delta_mu @ B
        
            try:
                u_rb_mu = npl.solve(B.T @ A_delta_mu @ B, B.T @ f_delta_mu)
            except npl.LinAlgError:
                u_rb_mu = npl.solve(B[:,np.newaxis].T @ A_delta_mu @ B[:,np.newaxis], 
                                    B[:,np.newaxis].T @ f_delta_mu)
            
            # Pour l'instant, eta(mu) = ||u_delta - u_rb||_L2
            eta[idx_mu] = npl.norm(u_delta_mu - u_rb_mu, ord=2)
        
        # On cherche la pire approximation
        idx_worst_mu = np.argmax(eta)
        
        B = np.vstack((B, U_DELTA[idx_worst_mu]))
        
        np.delete(MUs, idx_worst_mu)
        np.delete(U_DELTA, idx_worst_mu)
        
        err = eta[idx_worst_mu]
        n += 1
        
    return B, err

In [78]:
### computes greedy_base
T0 = np.array([1,50,50])
T1 = np.array([1,5,100])
training_set = training_set_creator(T0,T1)
Base, error = greedy_algorithm(0.0001, training_set)
# B, A, f = greedy_algorithm(0.0001, training_set)

Itération n°1
Itération n°2


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 34 is different from 2)

In [ ]:
### From Base, fast evaluates u_rb(mu_test)
mu_test = (0.28,40)
U_rb = reduced_solver(mu_test,A_q,shape)
X,U_true = solver(mu_test,plot = True)
plt.plot(X,U_true,label="true_sol")
plt.plot(X,Base.dot(U_rb),label="reduced_sol")
plt.legend()
plt.show()
print(np.sum(np.abs(U_true-Base.dot(U_rb))))

In [53]:
apply_mu([2.5, 0.5])[1].shape

(34,)

In [69]:
B.shape, A.shape, f.shape

((34,), (34, 34), (34,))

In [72]:
B[:,np.newaxis].shape

(34, 1)

In [71]:
B.shape

(34,)